###Packages 

In [ ]:

# ! pip install TextBlob

import json
import pandas as pd
import numpy as np
import string
import time

# Text polarity
from textblob import TextBlob

# For model-building
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer,HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

# SVD
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.preprocessing import StandardScaler

# Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# For text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

### Data 
Getting the.   data. First, we are going to get the data from the json. Then we are going to process it. And then use ML.

In [ ]:

# Read the json file
trainSet = pd.read_json("./train.json")
testSet = pd.read_json("./test.json")
trainSet.head(3)

We are going to remove the papedId and authorName because it won't tell us nothing about the author. 
(We already have authorId to identify the author, so let's remove authorName too).

In [ ]:
trainSet = trainSet.drop('authorName',axis = 1)
trainSet = trainSet.drop('paperId',axis = 1)
testSet = testSet.drop('paperId',axis = 1)

# Now, let's have a look to what we have:

# We take an example: The amount of data for authorId 3188285

print("Amount of data:",len(trainSet.loc[trainSet["authorId"]==3188285]))
# First instance:
print(trainSet.loc[trainSet["authorId"]==3188285].iloc[1])
# All the venues from this author
print(trainSet.loc[trainSet["authorId"]==3188285,["venue"]])

# So for every author we must somehow encode the title and the abstract. 
# Furthermore, we might deduce from the previous example that author likes to write about a topic related with venue "CLPsych". 
# Thus, we can consider splitting venues.

# Also, we can merge title and abstract. The "Type of writing" is the same.

# Merge title and abstract
trainSet["text"] = trainSet["title"] + '. '+ trainSet["abstract"]
trainSet = trainSet.drop('abstract',axis = 1)
trainSet = trainSet.drop('title',axis = 1)
# Same in test dataset
testSet["text"] = testSet["title"] + '. '+ testSet["abstract"]
testSet = testSet.drop('abstract',axis = 1)
testSet = testSet.drop('title',axis = 1)

